In [1]:
#!pip install keras-segmentation

import pandas as pd
import numpy as np
import scipy
import random
import os
import matplotlib.pyplot as plt
import seaborn
import cv2 as cv
import nibabel as nib
import tqdm
import gc
import warnings
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import losses, metrics
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from tensorflow.keras.layers import concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Multiply
from sklearn.model_selection import train_test_split
import keras_segmentation as ks

os.environ["SM_FRAMEWORK"]="tf.keras"
import segmentation_models as sm
warnings.filterwarnings('ignore')
%matplotlib inline
print("Version: ", tf.version.VERSION)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "not available")

Segmentation Models: using `tf.keras` framework.
Version:  2.1.0
GPU is available


In [2]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2


In [2]:
metadata = pd.read_csv('/home/deeplearning/Documents/DataSets/input/covid19-ct-scans/metadata.csv')

In [4]:
# define custom learning rate schedule
class CosineAnnealingLearningRateSchedule(callbacks.Callback):
    # constructor
    def __init__(self, n_epochs, n_cycles, lrate_max, verbose=0):
        self.epochs = n_epochs
        self.cycles = n_cycles
        self.lr_max = lrate_max
        self.lrates = list()
 
    # calculate learning rate for an epoch
    def cosine_annealing(self, epoch, n_epochs, n_cycles, lrate_max):
        epochs_per_cycle = np.floor(n_epochs/n_cycles)
        cos_inner = (np.pi * (epoch % epochs_per_cycle)) / (epochs_per_cycle)
        return lrate_max/2 * (np.cos(cos_inner) + 1)
 
    # calculate and set learning rate at the start of the epoch
    def on_epoch_begin(self, epoch, logs=None):
        # calculate learning rate
        lr = self.cosine_annealing(epoch, self.epochs, self.cycles, self.lr_max)
        # set learning rate
        K.set_value(self.model.optimizer.lr, lr)
        # log value
        self.lrates.append(lr)

In [5]:
batch_size = 5
epochs = 60
lrmax = 0.01
n_cycles = 5
lr_cb = CosineAnnealingLearningRateSchedule(epochs, n_cycles, lrmax)

In [6]:
img_size = 512

In [7]:
from Models import Attention_ResUNet, UNet, Attention_UNet
import tensorflow_addons as tfa

with strategy.scope():
    att_res_model = sm.FPN('resnet34', encoder_weights=None, input_shape=(512, 512, 2), classes=1, activation='sigmoid')
    #Model_1 = cts_model((img_size, img_size, 1))
    #att_res_model = Attention_ResUNet((img_size, img_size, 1), dropout_rate=0.3)
#     att_res_model = generate_unet_512()
    dice_loss = sm.losses.DiceLoss()
    focal_loss = sm.losses.CategoricalFocalLoss()
    total_loss = dice_loss + (1 * focal_loss)
    metrics = [sm.metrics.IOUScore(), sm.metrics.FScore()]
    #att_res_model.compile(optimizer=optimizers.Adam(lr = 1e-2), loss=TverskyLoss, metrics=[ tf.keras.metrics.MeanIoU(num_classes=2)])
    att_res_model.compile(optimizer=optimizers.Adam(lr = 1e-2), loss=tf.keras.losses.BinaryCrossentropy(), metrics=metrics)
#att_res_model.summary()

NameError: name 'strategy' is not defined

In [ ]:
#tf.keras.utils.plot_model(Model_1, show_shapes=True)

In [ ]:
checkpoint_fpath = "Modelsss/Test_best.hdf5"
cts_checkpoint_cb = callbacks.ModelCheckpoint(checkpoint_fpath, 
                                              monitor='val_iou_score', 
                                              save_best_only=True, 
                                              mode='max', 
                                              verbose=1,
                                              save_weights_only=True)

In [ ]:
def get_hu(Test):
        image = Test.pixel_array.astype(np.int16)

        intercept = Test.RescaleIntercept
        slope = Test.RescaleSlope

        if slope != 1:
            image = slope * image.astype(np.float64)
            image = image.astype(np.int16)

        image += np.int16(intercept)

        return np.array(image, dtype=np.int16)

In [ ]:
import os
from pydicom import dcmread
# directory='/home/deeplearning/Documents/DataSets/COD0001'
directory='/home/deeplearning/Documents/DataSets/Patch-Test'
patients = os.listdir(directory)
patients.sort(key= lambda x: (len (x), x))
    
while patients[0][0]=='.':
    del patients[0]
        
Slices = [dcmread(directory + '/' + s, force=True) for s in patients]

In [ ]:
image_path='/home/deeplearning/Documents/DataSets/npy/Augmented_norm_every_slice_fixed/image/'
mask_path='/home/deeplearning/Documents/DataSets/npy/Augmented_norm_every_slice_fixed/mask/'

In [ ]:
len(os.listdir(image_path))

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set=train_test_split(os.listdir(image_path), test_size = 0.10, random_state = 0)
train_set, val_set=train_test_split(train_set, test_size = 0.11, random_state = 0)

In [ ]:
len(test_set)

In [ ]:
clahe = cv.createCLAHE(clipLimit=3.0)

def norm(CT):
    norm_data = (CT)/(255)
    return norm_data

def normalize(CT):
    Min=-1000
    Max=400
    CT=np.clip(CT, Min, Max)
    norm_data = (CT-Min)/(Max-Min)
    return norm_data

In [ ]:
class Custom_Generator(tf.keras.utils.Sequence):
    
    def __init__(self, images, masks, batch_size=32):
        'Initialization'
        self.batch_size = batch_size
        self.images = images
        self.masks = masks

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(len(self.images) / self.batch_size)

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        IDs = self.images[index*self.batch_size:(index+1)*self.batch_size]
        
#         image_set=np.array([np.load(image_path + ID) for ID in IDs])
#         image_set=np.array([np.repeat(np.load(image_path + ID), 3, axis=-1) for ID in IDs])
        image_set=[np.repeat(np.load(image_path + ID), 2, axis=-1) for ID in IDs]
#         image_set=np.array([norm(clahe.apply(np.uint8(np.load(image_path + ID)*255))) for ID in IDs])
        
        for raw in image_set:
            raw[...,1]= norm(clahe.apply(np.uint8(raw[...,1]*255)))


            
        image_set = np.array(image_set)
        
        mask_set=np.array([np.load(mask_path + ID) for ID in IDs])
#         mask_set=np.array([tf.keras.utils.to_categorical(np.load(mask_path + ID), num_classes=4) for ID in IDs])
#         mask_set=np.array([tf.keras.utils.to_categorical(np.where(np.load(mask_path + ID)==3, 2, np.load(mask_path + ID)), num_classes=3) for ID in IDs])
        
        return image_set, mask_set


In [ ]:
train_generator=Custom_Generator(train_set, train_set, batch_size=batch_size)
val_generator=Custom_Generator(val_set, val_set, batch_size=batch_size)
test_generator=Custom_Generator(test_set, test_set, batch_size=batch_size)

In [ ]:

    
ctsRes = att_res_model.fit(train_generator, 
                       steps_per_epoch= int(len(train_set) / batch_size), 
                       epochs=epochs,
                       verbose=1,
                       validation_data=val_generator,
                        validation_steps=int(len(val_set) / batch_size),
                        validation_freq=1,
                        callbacks = [cts_checkpoint_cb, lr_cb]
                       )
    
att_res_model.save('Modelsss/Test.h5')
    
gc.collect()

In [ ]:
plt.style.use('ggplot')
fig, axes = plt.subplots(1, 1, figsize=(11,5))
axes.plot(ctsRes.history['loss'], color='b', label='train-loss')
axes.plot(ctsRes.history['val_loss'], color='m', label='valid-loss')
axes.plot(ctsRes.history['iou_score'], color='y', label='iou_score')
axes.plot(ctsRes.history['val_iou_score'], color='r', label='val_iou_score')
axes.set_ylabel('loss')
axes.set_xlabel('epoch')
axes.set_ylim([0,1])
axes.legend();
plt.show()

In [ ]:
att_res_model.evaluate(test_generator)

In [ ]:
Test=np.load(image_path+'100.npy')
print(Test.shape)
len(np.unique(Test))

In [ ]:
Test.dtype

In [ ]:
np.max(Test)

In [ ]:
plt.imshow(Test)
plt.grid(False)

In [ ]:
# Test=np.repeat(Test, 3, axis=-1)
Test = clahe.apply(np.uint8(Test*255))
plt.imshow(Test, cmap='gray')
plt.grid(False)

In [ ]:
from scipy import ndimage as ndi
import cv2 as cv
from skimage.morphology import disk,erosion, dilation, opening, closing
import logging
logging.getLogger('tensorflow').disabled = True
end_check = False

Test_size=5
for i in range(15):
    
    if (i+1)*Test_size > len(Slices):
        Test=Slices[i*Test_size:]
        end_check = True
        
    else:
        Test=Slices[i*Test_size:(i+1)*Test_size]
    
    for ii in range(len(Test)):
#         Test[ii]=cv.resize(normalize(get_hu(Test[ii])), dsize = (img_size, img_size),interpolation = cv.INTER_AREA).astype('float32')
        Test[ii]=normalize(get_hu(Test[ii]))#.astype('float32')
        Test[ii]=np.expand_dims(Test[ii], -1)
        Test[ii]=np.repeat(Test[ii],2,-1)
        Test[ii][...,1]=norm(clahe.apply(np.uint8(Test[ii][...,1]*255)))
    
    
    Test = np.asarray(Test)
    infection_pred=att_res_model.predict(Test)
    
    
    fig, axes = plt.subplots(2, Test.shape[0], figsize=(24,9))
    for iii in range(Test.shape[0]):
        
        axes[0,iii].imshow(np.squeeze(Test[iii,...,0]), cmap='gray')
        axes[0,iii].set_title('Original CT')
        axes[0,iii].set_xticks([]); axes[0,iii].set_yticks([])

        axes[1,iii].imshow(np.squeeze(Test[iii,...,0]), cmap='gray')
#         axes[1,iii].imshow(np.squeeze(np.argmax(infection_pred[iii,...], axis= -1)), alpha=0.7)
        axes[1,iii].imshow(np.where(np.squeeze(infection_pred[iii,...])<0.1,0,1), alpha=0.7)

        axes[1,iii].set_title('Predicted infection')
        axes[1,iii].set_xticks([]); axes[1,iii].set_yticks([])
        
    plt.show()
    print('\n \n ################################################################################################')
    print(' ################################################################################################\n \n \n')
    
    if end_check:
        break